In [13]:
import xgboost as xgb
import numpy as np
import pandas as pd

In [14]:
# Metric
def mape(y_true, y_pred): 
    # Mean Absolute Percentage Error (mape)
    assert(min(y_true) > 0)
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    relative_error = np.abs((y_true-y_pred)/y_true)
    return np.sum(relative_error)/len(y_true)

In [15]:
import time
train = pd.read_csv('./unshuffled/train_unshuffled.csv', header=None)
test = pd.read_csv('./unshuffled/test_unshuffled.csv', header=None)
Xtrain = train[train.columns[:-1]]
ytrain = train[train.columns[-1]]
Xtest = test[test.columns[:-1]]
ytest = test[test.columns[-1]]

# One Hot Encoder
try:
    from category_encoders import OneHotEncoder
    ohe = OneHotEncoder()
except:
    print("Module category_encoders not available")

# Fit encoder
tic = time.perf_counter()
Xtrain = ohe.fit_transform(Xtrain, ytrain)
toc = time.perf_counter()
print("Fitting completed in ",round(toc-tic,2), "sec")
Xtest = ohe.transform(Xtest)

Fitting completed in  32.65 sec


In [ ]:
# Neural network
import tensorflow as tf
s = 1181
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(1000, activation='relu', input_shape=(s,)),
    tf.keras.layers.Dense(500, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss=tf.keras.losses.mean_absolute_error)
model.build()
print(model.summary())
model.fit(Xtrain.values, ytrain.values, epochs=10, batch_size=64)

In [ ]:
file = open('results.txt', 'a+') 
file.write(' NN: '+str(mape(ytest, np.concatenate(model.predict(Xtest.values), axis=0))))
file.close()

### XGBOOST 

In [21]:
# Sparsify 
from scipy.sparse import csr_matrix
Xtrain = csr_matrix(Xtrain)
Xtest = csr_matrix(Xtest)
Xtrain

<200000x1181 sparse matrix of type '<class 'numpy.longlong'>'
	with 1289030 stored elements in Compressed Sparse Row format>

In [5]:
dtrain = xgb.DMatrix(Xtrain, label=ytrain)
dtest = Xtest
cols = []
for i in range(1181):
    cols.append(str(i))
dtest.columns = cols
dtest = xgb.DMatrix(dtest, label=ytest)

In [6]:
# Parameters, taken from 'Entity Embeddings of Categorical Variables'
param = {'max_depth': 10, 'eta': 0.02, 'objective': 'reg:linear', 'colsample_bytree': 0.7, 'subsample': 0.7}
param['nthread'] = 8
param['eval_metric'] = 'mae'
param['verbosity'] = 1
evallist = [(dtest, 'eval'), (dtrain, 'train')]
# Training
num_round = 3000
bst = xgb.train(param, dtrain, num_round)#, early_stopping_rounds=100)#, evallist)
#bst.save_model('xgboost_model')

[17:11:42] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [7]:
# Load model if trained earlier
#bst = xgb.Booster({'nthread': 4})  # init model
#bst.load_model('xgboost_model')  # load data
# Evaluation on test data
file = open('results.txt', 'a+') 
file.write(' xgboost: '+str(mape(ytest, bst.predict(dtest)))+'\n')
file.close()

### KNN

In [8]:
from sklearn.neighbors import KNeighborsRegressor
KNN = KNeighborsRegressor(n_neighbors=10, weights='distance', p=1, n_jobs=-1)
KNN.fit(Xtrain, ytrain)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=-1, n_neighbors=10, p=1,
                    weights='distance')

In [9]:
file = open('results.txt', 'a+') 
file.write(' KNN: '+str(mape(ytest, KNN.predict(Xtest)))+'\n')
file.close()

### Random Forest

In [10]:
from sklearn.ensemble import RandomForestRegressor
RFR = RandomForestRegressor(n_estimators=200, max_depth=35, min_samples_split=2, min_samples_leaf=1, n_jobs=-1)
RFR.fit(Xtrain,ytrain)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=35, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=200, n_jobs=-1, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [11]:
file = open('results.txt', 'a+') 
file.write(' RFR: '+str(mape(ytest, RFR.predict(Xtest)))+'\n')
file.close()